In [19]:
import pandas as pd
import sqlite3 as sql

In [20]:
#Import the data base
data_base = sql.connect("/Users/alneagu/Downloads/sql-murder-mystery.db")

#Set the dataframe width 
pd.set_option("display.max_colwidth",None)

# Crime Report
<p> Find the crime report about the murder that took place on January 15 2018 in SQL City

In [21]:
query1 = '''
SELECT * 
FROM crime_scene_report
WHERE date = 20180115 AND type = "murder" AND city = "SQL City"
'''
pd.read_sql_query(query1, data_base)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

# Identify the witnesses


1. First witness lives at the last house on "Northwestern Dr"

In [22]:
query2 = ''' 
SELECT * 
FROM person
WHERE  address_street_name = "Northwestern Dr" 
ORDER BY address_number DESC LIMIT 1; 
'''
pd.read_sql_query(query2, data_base)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


2. Second witness name is Annabel and lives on Franklin Ave

In [23]:
query3 = ''' 
SELECT * 
FROM person
WHERE  address_street_name = "Franklin Ave" AND name LIKE "%Annabel%" 
'''
pd.read_sql_query(query3, data_base)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


# Interviews of the witnesses

In [24]:
query4 = ''' 
SELECT * 
FROM interview
WHERE person_id = 16371 OR person_id = 14887
'''

pd.read_sql_query(query4, data_base)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


> Based on the information from the witnesses, we should look further in the gym and driver license databases for the following details:
- membership number starting with "48Z"
- logs for the people working out on January the 9th in the time frame when Annabel Miller was there
- car plate of a man that included "H42W" 

### Gym database

- Check for the people who have a gold membership and their membership number starts with "48Z"

In [28]:
query5 = '''
SELECT name, membership_status,membership_id , person_id  
FROM get_fit_now_member
JOIN get_fit_now_check_in 
ON get_fit_now_check_in.membership_id = get_fit_now_member.id
WHERE check_in_date = 20180109 AND membership_status = "gold" AND membership_id LIKE "48Z%"
'''

pd.read_sql_query(query5, data_base)

,name,membership_status,membership_id,person_id
0,Joe Germuska,gold,48Z7A,28819
1,Jeremy Bowers,gold,48Z55,67318


There are two people that match the above requirements:
1. Joe Germuska
2. Jeremy Bowers

> Check which one of the two males is the killer using the information from the driver license database
### Driver license database

In [41]:
query6 = '''
SELECT *
FROM person
JOIN get_fit_now_member AS gym
  ON gym.person_id = person.id
JOIN drivers_license dl
  ON person.license_id = dl.id
WHERE dl.plate_number LIKE "%H42W%" AND (gym.person_id = 28819 OR gym.person_id = 67318)
 '''

pd.read_sql_query(query6, data_base)

,id,name,license_id,address_number,address_street_name,ssn,id,person_id,name,membership_start_date,membership_status,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,48Z55,67318,Jeremy Bowers,20160101,gold,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


> We found that the murdered is **Jeremny Bowers**.
> We can check now if the answer is correct on the website 

INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

> "Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [42]:
query7 = ''' 
SELECT * 
FROM interview
WHERE person_id = 67318
'''
pd.read_sql_query(query7, data_base)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [59]:
query8 = '''
SELECT person.name, COUNT(*) AS no_of_concerts_attended, fb.event_name
FROM person
JOIN drivers_license dl
  ON person.license_id = dl.id
JOIN facebook_event_checkin fb
  ON person.id = fb.person_id
WHERE dl.gender = "female" 
AND dl.height BETWEEN 65 AND 67
AND dl.hair_color = "red"
AND dl.car_make = "Tesla"
AND dl.car_model ="Model S"
AND fb.event_name = "SQL Symphony Concert"
 '''

pd.read_sql_query(query8, data_base)

,name,no_of_concerts_attended,event_name
0,Miranda Priestly,3,SQL Symphony Concert


Check on the website if Miranda Priestly was behind the murder
> INSERT INTO solution VALUES (1, 'Miranda Priestly');
SELECT value FROM solution;

> "Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"